#### Libraries

In [1]:
%%javascript
utils.load_extension("collapsible_headings/main")
utils.load_extension("hide_input/main")
utils.load_extension("autosavetime/main")
utils.load_extension("execute_time/ExecuteTime")
utils.load_extension("code_prettify/code_prettify")
utils.load_extension("scroll_down/main")
utils.load_extension("jupyter-js-widgets/extension")

<IPython.core.display.Javascript object>

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
from tqdm import trange

plt.style.use("seaborn")
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error

from nnet import ReadDataset, Net,ResNet
import time
from loss_functions import interval_score_loss

from pytorch_tabnet.tab_model import TabNetClassifier
from gauss_rank_scaler import GaussRankScaler
tic = time.time()
from models.lgbm import (compute_metrics, preprocess)

from tools.metrics import (
    get_avg_volumes,
)
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

In [17]:

def plot_feature_importance(
    columnas, model_features, columns_ploted=10, model_name="Catboost"
):
    """
    This method is yet non-tested
    
    This function receives a set of columns feeded to a model, and the importance of each of feature.
    Returns a graphical visualization
    
    Call it fot catboost pipe example:
    plot_feature_importance(pipe_best_estimator[:-1].transform(X_tr).columns,pipe_best_estimator.named_steps['cb'].get_feature_importance(),20)
    
    Call it for lasso pipe example:
    plot_feature_importance(pipe_best_estimator[:-1].transform(X_tr).columns,np.array(pipe_best_estimator.named_steps['clf'].coef_.squeeze()),20)
    """

    feature_importance = pd.Series(index=columnas, data=np.abs(model_features))
    n_selected_features = (feature_importance > 0).sum()
    print(
        "{0:d} features, reduction of {1:2.2f}%".format(
            n_selected_features,
            (1 - n_selected_features / len(feature_importance)) * 100,
        )
    )
    plt.figure()
    feature_importance.sort_values().tail(columns_ploted).plot(
        kind="bar", figsize=(18, 6)
    )
    plt.title("Feature Importance for {}".format(model_name))
    plt.show()



In [18]:
def my_metric(pred, lower, upper):

    metric_pair = compute_metrics(
        preds=pred,
        lower=lower,
        upper=upper,
        y=val_y_raw,
        offset=val_offset,
        X=val_x_orig,
        avg_volumes=avg_volumes,
    )
    return metric_pair[0],metric_pair[1]

In [19]:
def postprocess_submission(submission_df, solve_submission_issues=True):

    join_on = ["country", "brand", "month_num"]
    keep = join_on + ["volume"]

    df_vol = pd.read_csv("../data/gx_volume.csv").loc[:, keep]

    both_ds = submission_df.merge(
        df_vol,
        on=join_on,
        how="left",
    )

    both_ds.loc[both_ds["volume"].notnull(), "prediction"] = both_ds[both_ds["volume"].notnull()]["volume"].values
    both_ds.loc[both_ds["volume"].notnull(), "pred_95_high"] = both_ds[both_ds["volume"].notnull()]["volume"].values + 0.01
    both_ds.loc[both_ds["volume"].notnull(), "pred_95_low"] = both_ds[both_ds["volume"].notnull()]["volume"].values - 0.01

    final_cols = join_on + ["pred_95_low", "prediction", "pred_95_high"]

    final_df =  both_ds.loc[:, final_cols]

    if solve_submission_issues:

        if (final_df.pred_95_low > final_df.pred_95_high).any():
            raise("Stop please, upper < lower")

        cond_lower_mean = final_df.pred_95_low > final_df.prediction
        if cond_lower_mean.any():
            print("Solving lower > mean")
            final_df.loc[cond_lower_mean, "prediction"] = \
                final_df.loc[cond_lower_mean, "pred_95_low"] + 0.01

        cond_upper_mean = final_df.prediction > final_df.pred_95_high
        if cond_upper_mean.any():
            print("Solving upper < mean")
            final_df.loc[cond_upper_mean, "prediction"] = \
                final_df.loc[cond_upper_mean, "pred_95_high"] - 0.01


    return final_df


In [20]:
def my_metric(pred, lower, upper):

    metric_pair = compute_metrics(
        preds=pred,
        lower=lower,
        upper=upper,
        y=val_y_raw,
        offset=val_offset,
        X=val_x_orig,
        avg_volumes=avg_volumes,
    )
    return metric_pair[0],metric_pair[1]

In [21]:
file_name = "linear_base"
offset_name = "last_before_3_after_0"
full_df = pd.read_csv("../data/gx_merged_lags_months.csv")
    # volume_features = pd.read_csv("data/volume_features.csv")
submission_df = pd.read_csv("../data/submission_template.csv")
train_tuples = pd.read_csv("../data/train_split.csv")
valid_tuples = pd.read_csv("../data/valid_split.csv")

 
# full_df = full_df.merge(volume_features, on=["country", "brand"])


full_df["volume_offset"] = (full_df["volume"] - full_df[offset_name]) / full_df[offset_name]
full_df = preprocess(full_df)

test_df = full_df[full_df.test == 1].copy().reset_index(drop=True)

real_full = full_df.copy()

full_df = full_df[full_df.test == 0]

train_df = full_df.merge(train_tuples, how="inner").reset_index(drop=True)
val_df = full_df.merge(valid_tuples, how="inner").reset_index(drop=True)

    # TODO: no need for calculation every time
avg_volumes = get_avg_volumes()

to_drop = ["volume", "volume_offset",'test','A','country_mean_before_24_after_0']
categorical_cols = [
        "country", "brand", "therapeutic_area", "presentation", "month_name",
        "month_country", "month_presentation", "month_area",
        "month_country_num", "month_presentation_num", "month_area_num",
        "month_month_num"
    ]

    # Prep data
train_x = train_df.drop(columns=to_drop)
train_y = train_df.volume_offset
train_offset = train_df[offset_name]



full_x = full_df.drop(columns=to_drop)
full_y = full_df.volume_offset
full_offset = full_df[offset_name]

real_full_x = real_full.drop(columns=to_drop)


val_x = val_df.drop(columns=to_drop)
val_x_orig = val_x
val_y = val_df.volume_offset
val_y_raw = val_df.volume
val_offset = val_df[offset_name]

test_x = test_df.drop(columns=to_drop)
test_offset = test_df[offset_name]

## Single Model

In [22]:
from category_encoders.one_hot import OneHotEncoder

In [23]:
# Prep pipeline
#te = TargetEncoder(cols=categorical_cols)

#te_residual = TargetEncoder(cols=categorical_cols)


In [43]:
categorical_cols = [
        "country", "brand", "therapeutic_area", "presentation", "month_name",
        "month_country", "month_presentation", "month_area",
        "month_country_num", "month_presentation_num", "month_area_num",
        "month_month_num"
    ]
tec = ['month_month_num','month_area_num','month_presentation_num','month_country_num']
one = list(set(categorical_cols) - set(tec))

In [44]:
te = TargetEncoder(cols=tec)
ohe = TargetEncoder(cols=one)

In [47]:
len(one)

8

In [49]:
len(tec)

4

In [50]:
len(categorical_cols)

12

In [39]:
lgbm = LGBMRegressor(n_jobs=-1, n_estimators=100, objective="quantile", alpha=0.5,)


pipe = Pipeline([("te", te),('ohe',ohe) ,("lgb", lgbm)])

# Fit cv model
pipe.fit(train_x, train_y)

/opt/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: country, brand, month_name, therapeutic_area, presentation, month_country, month_presentation, month_area

In [12]:
pp = pipe.predict(val_x)
my_metric(pp, pp - 0.1 * pp, pp + 0.1 * pp)

(13.9298589697328, 123.25719917969023)

In [ ]:
pp = X_aux.forecast
my_metric(pp, pp - 0.1 * pp, pp + 0.1 * pp)

In [79]:
pipes = {}
lgbms = {}
tes = {}
preds = {}
preds_test = {}
res = []
train_df_aux = train_df.copy()
val_df_aux = val_df

for i in trange(0,24):

    
    # Extract Train
    X_train = train_df_aux[train_df_aux.month_num == i].drop(columns=to_drop)
    y_train = train_df_aux[train_df_aux.month_num == i].volume_offset


    
   # Extract validation
    X_val = val_df_aux[val_df_aux.month_num == i].drop(columns=to_drop)
    y_val = val_df_aux[val_df_aux.month_num == i].volume
    
    
    # Instantiate and fit model
    tes[i] = OneHotEncoder(cols=categorical_cols)
    lgbms[i] = LGBMRegressor(n_jobs=-1, n_estimators=200, objective="quantile")

    pipes[i] = Pipeline([("te", tes[i]), ("lgb", lgbms[i])])
    pipes[i].fit(X_train, y_train)

    # Make predictions
    preds[i] = pipes[i].predict(X_val)


    X_val['forecast'] = pipes[i].predict(X_val)
    
    # Add new column in train
    pred_m = 'pred_month_'+str(i)
    train_df_aux[pred_m] = pipes[i].predict(train_df_aux.drop(columns=to_drop))
    
    # Add new column in validation
    pred_m = 'pred_month_'+str(i)
    val_df_aux[pred_m] = pipes[i].predict(val_df_aux.drop(columns=to_drop))
    
    # Store predictions in auxiliar array
    if i==0:
        X_aux = X_val.copy()
    else:
        X_aux = X_aux.append(X_val)
        
        
        

    plot_feature_importance(X_train.columns,pipes[i].named_steps['lgb'].feature_importances_)

  0%|          | 0/24 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
  0%|          | 0/24 [00:04<?, ?it/s]


ValueError: Unexpected input dimension 77, expected 53

In [29]:

upper_bounds = [0.8, 1.]
lower_bounds = [1., 1.2]

min_unc = 1e8
best_upper_bound = 0
best_lower_bound = 0

In [36]:
bounds = [0, 0.01, 0.1, 0.5, 1, 1.5]


min_unc = 1e8
best_bound = 0
for bound in bounds:
    print(f"Bound: {bound}")
    metric_pair = compute_metrics(
        X_aux.forecast,
        X_aux.forecast -1,
        X_aux.forecast +1,
        y_val,
        X_val,
        avg_volumes,
    )
    print(metric_pair)

    unc_metric = metric_pair.values[1]

    if unc_metric < min_unc:
        min_unc = unc_metric
        best_bound = bound

Bound: 0


AssertionError: We should have 24 sorted actuals, upper_bound and lower_bound intervals

In [ ]:


print(min_unc)
print(best_upper_bound)
print(best_lower_bound)

In [ ]:
submission_df["pred_95_low"] = (preds_test[0.25] * best_lower_bound + 1) * test_offset
submission_df["pred_95_high"] = (preds_test[0.75] * best_upper_bound + 1) * test_offset
submission_df["prediction"] = pipes[0.5].predict(test_x)

submission_df = postprocess_submission(submission_df)
submission_df.to_csv("submissions/baseline_relative_postprocess.csv", index=False)
